<a href="https://colab.research.google.com/github/secutron/TesTime/blob/main/ig_RC_osmu_v001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RC(Randomly Controled) 컨셉 테스트



In [1]:
from datetime import datetime
print("Current Date and Time: ", datetime.now())

Current Date and Time:  2021-05-11 22:57:31.605623


## Ignite 준비

In [2]:
!pip install pytorch-ignite

     |████████████████████████████████| 204kB 5.4MB/s 


In [3]:
import os

in_colab = "COLAB_TPU_ADDR" in os.environ
with_torch_launch = "WORLD_SIZE" in os.environ

if in_colab:
    # https://github.com/pytorch/builder/pull/750
    VERSION = "20200607" 
    #VERSION = "nightly"
    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  21863      0 --:--:-- --:--:-- --:--:-- 21770
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200607 ...
     |████████████████████████████████| 61kB 3.6MB/s 
Uninstalling torch-1.8.1+cu101:
ERROR: earthengine-api 0.1.260 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
Done updating TPU runtime
  Successfully uninstalled torch-1.8.1+cu101
Uninstalling torchvision-0.9.1+cu101:
  Successfully uninstalled torchvision-0.9.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200607-cp37-cp37m-linux_x86_64.whl...
\ [1 file

In [4]:
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset

from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, models
from torchvision.transforms import Compose, Normalize, Pad, RandomCrop, RandomHorizontalFlip, ToTensor

import ignite.distributed as idist
from ignite.contrib.engines import common
from ignite.contrib.handlers import ProgressBar
from ignite.engine import Engine, Events, create_supervised_evaluator
from ignite.metrics import Accuracy
from ignite.utils import manual_seed, setup_logger

In [6]:
def get_MNIST(path="./"):
    input_size = 28
    num_classes = 10
    
    transform = Compose(
        [ToTensor(), Normalize((0.1307,), (0.3081,))]
        )

    train_dataset = datasets.MNIST(path + "data/", train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(path + "data/", train=False, download=True, transform=transform)

    return input_size, num_classes, train_dataset, test_dataset


def get_FashionMNIST(path="./"):
    input_size = 28
    num_classes = 10

    transform = Compose(
        [ToTensor(), Normalize((0.2861,), (0.3530,))]
        )

    train_dataset = datasets.FashionMNIST(path + "data/", train=True, download=True, transform=transform)
    test_dataset = datasets.FashionMNIST(path + "data/", train=False, download=True, transform=transform
                                         )
    return input_size, num_classes, train_dataset, test_dataset


def get_SVHN(path="./"):
    input_size = 32
    num_classes = 10

    transform = Compose(
        [ToTensor(), Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        )

    train_dataset = datasets.SVHN(path + "data/SVHN", split="train", transform=transform, download=True)
    test_dataset = datasets.SVHN(path + "data/SVHN", split="test", transform=transform, download=True)

    return input_size, num_classes, train_dataset, test_dataset


def get_CIFAR10(path="./"):
    input_size = 32
    num_classes = 10
    train_transform = Compose(
        [
            #            Pad(4),
            RandomCrop(32, fill=128, padding=4),
            RandomHorizontalFlip(),
            ToTensor(),
            Normalize((0.485, 0.456, 0.406), (0.229, 0.23, 0.225)),
        ]
        )
    
    train_dataset = datasets.CIFAR10(path + "data/CIFAR10", train=True, transform=train_transform, download=True)

    test_transform = Compose(
        [
            ToTensor(),
            Normalize((0.485, 0.456, 0.406), (0.229, 0.23, 0.225)),
        ]
        )
    
    test_dataset = datasets.CIFAR10(path + "data/CIFAR10", train=False, transform=test_transform, download=True)

    return input_size, num_classes, train_dataset, test_dataset


def get_notMNIST(path="./"):
    input_size = 28
    num_classes = 10

    transform = Compose(
        [ToTensor(), Normalize((0.4254,), (0.4586,))]
        )

    test_dataset = NotMNIST(path + "data/", transform=transform)

    return input_size, num_classes, None, test_dataset

In [7]:
all_datasets = {
    "MNIST": get_MNIST,
    "notMNIST": get_notMNIST,
    "FashionMNIST": get_FashionMNIST,
    "SVHN": get_SVHN,
    "CIFAR10": get_CIFAR10,
}

In [8]:
class NotMNIST(Dataset):
    def __init__(self, root, transform=None):
        root = os.path.expanduser(root)

        self.transform = transform

        data_dict = loadmat(os.path.join(root, "notMNIST_small.mat"))

        self.data = torch.tensor(
            data_dict["images"].transpose(2, 0, 1), dtype=torch.uint8
        ).unsqueeze(1)

        self.targets = torch.tensor(data_dict["labels"], dtype=torch.int64)

    def __getitem__(self, index):
        img = self.data[index]
        target = self.targets[index]

        if self.transform is not None:
            img = Image.fromarray(img.squeeze().numpy(), mode="L")
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self.data)


class FastFashionMNIST(datasets.FashionMNIST):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.data = self.data.unsqueeze(1).float().div(255)
        self.data = self.data.sub_(0.2861).div_(0.3530)

        self.data, self.targets = self.data.to("cuda"), self.targets.to("cuda")

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], self.targets[index]

        return img, target

In [9]:
def get_model(name):
    if name in models.__dict__:
        fn = models.__dict__[name]
    else:
        raise RuntimeError(f"Unknown model name {name}")

    return fn(num_classes=10)

In [10]:
def get_dataflow(config):

    if idist.get_rank() > 0: # 마스터 노드만 데이터 다운로드
        idist.barrier()

    dataset_name = config.get("dataset", "cifar10")
    if dataset_name == "cifar10":
        #train_dataset, test_dataset = get_train_test_datasets(config.get("data_path", "."))
        input_size, num_classes, train_dataset, test_dataset = all_datasets["CIFAR10"]()
    else: 
        # never happen.
        raise RuntimeError(f"Unknown dataset_name {dataset_name}")

    if idist.get_rank() == 0: # 마스터 노드만 데이터 다운로드 
        idist.barrier()

    train_loader = idist.auto_dataloader(
        train_dataset,
        batch_size=config.get("batch_size", 512),
        num_workers=config.get("num_workers", 8),
        shuffle=True,
        drop_last=True,
    )
    config["num_iters_per_epoch"] = len(train_loader)

    test_loader = idist.auto_dataloader(
        test_dataset,
        batch_size=2 * config.get("batch_size", 512),
        num_workers=config.get("num_workers", 8),
        shuffle=False,
    )
    return train_loader, test_loader

In [11]:
def initialize(config):
    
    model = get_model(config["model"])
    model = idist.auto_model(model)

    optimizer = optim.SGD(
        model.parameters(),
        lr=config.get("learning_rate", 0.1),
        momentum=config.get("momentum", 0.9),
        weight_decay=config.get("weight_decay", 1e-5),
        nesterov=True,
    )
    optimizer = idist.auto_optim(optimizer)
    criterion = nn.CrossEntropyLoss().to(idist.device())

    le = config["num_iters_per_epoch"]
    lr_scheduler = StepLR(optimizer, step_size=le, gamma=0.9)

    return model, optimizer, criterion, lr_scheduler

In [12]:
def create_trainer(model, optimizer, criterion, lr_scheduler, config):

    def train_step(engine, batch):
        x, y = batch[0].to(idist.device()), batch[1].to(idist.device())

        model.train()
        y_pred = model(x)
        loss = criterion(y_pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        return loss.item()

    trainer = Engine(train_step)

    if idist.get_rank() == 0:
        @trainer.on(Events.ITERATION_COMPLETED(every=200))
        def save_checkpoint():
            fp = Path(config.get("output_path", "output")) / "checkpoint.pt"
            torch.save(model.state_dict(), fp)

        # 진행 표시
        ProgressBar().attach(trainer, output_transform=lambda x: {"batch loss": x})

    return trainer

In [33]:
def training(local_rank, config):
    # spawn 된 경우를 생각하고 또 생각할 것


    # 모든 프로세스에서 한번씩 표시
    print(idist.get_rank(), ": run with config:", config, "- backend=", idist.backend())
    
    # 글로벌 랭크로 연결된 로거에 표시
    logger = setup_logger("ig_RC_osmu", distributed_rank=idist.get_rank())
    logger.info(f"rank: {idist.get_rank()}")

    # 로컬 랭크(해당 노드에서의 프로세스 id)에 연결된 로거에 표시
    # 단일 노드인 경우에는!!! 
    logger2 = setup_logger("ig_RC_osmu")
    logger2.info(f"local rank: '{idist.get_local_rank()}")


    # 데이터 로더 처리
    train_loader, val_loader = get_dataflow(config)
    model, optimizer, criterion, lr_scheduler = initialize(config)

    # 트레이너, 이밸류에이터 처리
    trainer = create_trainer(model, optimizer, criterion, lr_scheduler, config)
    evaluator = create_supervised_evaluator(model, metrics={"accuracy": Accuracy()}, device=idist.device())


    # 에폭 관련 이벤트
    @trainer.on(Events.EPOCH_COMPLETED(every=3))
    def evaluate_model():
        state = evaluator.run(val_loader)
        if idist.get_rank() == 0:
            print(f"eval metric: {state.metrics}")
    '''
    # 프로파일 - 싱글 에폭 소요
    @trainer.on(Events.EPOCH_COMPLETED)
    def log_epoch_time():
        print(f"rank: {idist.get_rank()}, epoch: {trainer.state.epoch} - {trainer.state.times['EPOCH_COMPLETED']}")
    '''
    @trainer.on(Events.COMPLETED)
    # 프로파일 - 전체 학습 소요
    def log_total_time():
        print(f"rank: {idist.get_rank()}, total: {trainer.state.times['COMPLETED']}")

    '''
    # 베이직/핸들러 타임 프로파일링
    if config["profile"] is True:
        #from ignite.contrib.handlers import BasicTimeProfiler
        #profiler = BasicTimeProfiler()

        from ignite.contrib.handlers import HandlersTimeProfiler
        profiler = HandlersTimeProfiler()

        profiler.attach(trainer)

        @trainer.on(Events.EPOCH_COMPLETED(every=3))
        def log_intermediate_results():
            profiler.print_results(profiler.get_results())
    '''

    # 로그 처리
    if idist.get_rank() == 0:
        tb_logger = common.setup_tb_logging(
            config.get("output_path", "output"), trainer, optimizer, evaluators={"validation": evaluator},
        )

    # 트레이터 실행
    trainer.run(train_loader, max_epochs=config.get("max_epochs", 3))

    if idist.get_rank() == 0:
        tb_logger.close()

In [36]:
import tensorflow as tf

print("\n")
print("*************************************")
print("Tensorflow version " + tf.__version__)
print("\n")


try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the comment above for instructions!')

print("\n*** connect to cluster")
tf.config.experimental_connect_to_cluster(tpu)

print("\n*** initialize tpu")
tf.tpu.experimental.initialize_tpu_system(tpu)

# tpu 0번부터 7번까지 배정 : thank Google!!!
# host(master)로 cpu 0번, worker로 다시 cpu 0번, worker에 딸린 tpu가 0번부터 7번으로 8개
print("\n*** tpu strategy")
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)



*************************************
Tensorflow version 2.4.1


Running on TPU  ['10.105.20.242:8470']

*** connect to cluster

*** initialize tpu


INFO:tensorflow:Initializing the TPU system: grpc://10.105.20.242:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.105.20.242:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [35]:
# 멀티노드 launch 유틸: 각 nodes에서 실행
# https://pytorch.org/ignite/v0.4.4.post1/distributed.html#ignite-distributed-launcher

print("\n")
print("*************************************")
print("Ignite : 멀티노드 경우는 고려되지 않음 by hkim 11MAY21")
print("\n")

config = {
    "profile": True,
    "model": "resnet18",
    "dataset": "cifar10",
}

# xxx
if __name__ == "__main__" and not (in_colab or with_torch_launch):

    backend = None  # or "nccl", "gloo", "xla-tpu" ...
    nproc_per_node = None  # or N to spawn N processes

    with idist.Parallel(backend=backend, nproc_per_node=nproc_per_node) as parallel:
        parallel.run(training, config)


# GPU
if __name__ == "__main__" and with_torch_launch:

    backend = "nccl"  # or "nccl", "gloo", "xla-tpu" ...
    nproc_per_node = None  # or N to spawn N processes

    with idist.Parallel(backend=backend, nproc_per_node=nproc_per_node) as parallel:
        parallel.run(training, config)


# TPU
if in_colab:
    
    backend = "xla-tpu"  # or "nccl", "gloo", "xla-tpu" ...
    #nproc_per_node = 1  # xxx
    nproc_per_node = 8  # 8 TPU (4칩 x 2코어까지 지원)

    # Parallel: Distributed launcher context manager
    # https://pytorch.org/ignite/distributed.html#ignite-distributed-launcher
    # Multi-node option : 2 nodes with 8 GPUs each 참고할 것
    # 예: python -m torch.distributed.launch --nnodes=2 --node_rank=0 --master_addr=master                 
    #     --master_port=3344 --nproc_per_node=8 --use_env main.py
    # spawn param이 설정되는 경우 확인할 것.

    from time import time
    start = time()
    with idist.Parallel(backend=backend, nproc_per_node=nproc_per_node) as parallel:
        parallel.run(training, config)
    end = time()
    print(f"Elapsed time: {end-start}")


2021-05-12 00:46:03,725 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'xla-tpu'
2021-05-12 00:46:03,727 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 8
	nnodes: 1
	node_rank: 0
2021-05-12 00:46:03,730 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7fccb3d015f0>' in 8 processes




*************************************
Ignite : 멀티노드 경우는 고려되지 않음 by hkim 11MAY21


0 : run with config: {'profile': True, 'model': 'resnet18', 'dataset': 'cifar10'} - backend= xla-tpu
1 : run with config: {'profile': True, 'model': 'resnet18', 'dataset': 'cifar10'} - backend= xla-tpu
5 : run with config: {'profile': True, 'model': 'resnet18', 'dataset': 'cifar10'} - backend= xla-tpu
3 : run with config: {'profile': True, 'model': 'resnet18', 'dataset': 'cifar10'} - backend= xla-tpu
4 : run with config: {'profile': True, 'model': 'resnet18', 'dataset': 'cifar10'} - backend= xla-tpu
7 : run with config: {'profile': True, 'model': 'resnet18', 'dataset': 'cifar10'} - backend= xla-tpu
6 : run with config: {'profile': True, 'model': 'resnet18', 'dataset': 'cifar10'} - backend= xla-tpu
2 : run with config: {'profile': True, 'model': 'resnet18', 'dataset': 'cifar10'} - backend= xla-tpu


2021-05-12 00:46:49,496 ig_RC_osmu INFO: rank: 0
2021-05-12 00:46:49,516 ig_RC_osmu INFO: local rank: '0


Files already downloaded and verified
Files already downloaded and verified


2021-05-12 00:46:51,931 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset 'Dataset CIFAR10': 
	{'batch_size': 64, 'num_workers': 1, 'drop_last': True, 'sampler': <torch.utils.data.distributed.DistributedSampler object at 0x7fccb3cafc90>, 'pin_memory': False}
2021-05-12 00:46:51,978 ignite.distributed.auto.auto_dataloader INFO: DataLoader is wrapped by `MpDeviceLoader` on XLA
2021-05-12 00:46:52,002 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset 'Dataset CIFAR10': 
	{'batch_size': 128, 'num_workers': 1, 'sampler': <torch.utils.data.distributed.DistributedSampler object at 0x7fccb3caf850>, 'pin_memory': False}
2021-05-12 00:46:52,014 ignite.distributed.auto.auto_dataloader INFO: DataLoader is wrapped by `MpDeviceLoader` on XLA


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


rank: 2, total: 108.41035032272339
rank: 1, total: 107.66651630401611
rank: 3, total: 107.72936391830444
rank: 7, total: 107.87349462509155
rank: 6, total: 108.32522249221802
eval metric: {'accuracy': 0.5149}
rank: 4, total: 107.7634961605072
rank: 5, total: 108.06047916412354
rank: 0, total: 120.23467087745667


2021-05-12 00:48:54,762 ignite.distributed.launcher.Parallel INFO: End of run


Elapsed time: 171.03995394706726


In [ ]:
%load_ext tensorboard
%tensorboard --logdir=output